In [3]:
import pandas as pd 
path ="C:/Users/catal/Downloads/collaborations-2025-delirium-visualization-main/collaborations-2025-delirium-visualization-main/data/2022/07/corrections_report_2022-07-27.html"
# Ubicación de un archivo o en el sistema de archivos

tablas = pd.read_html(path, index_col=0) 
# Función que extrae tablas de un archivo HTML y las devuelve como una lista de DataFrames.

In [4]:
tablas[0] 

,File info
Delay line number,1
Fit along Horizontal axis,First order (LINEAR)
Fit along Vertical axis,Second order (PARABOLIC)
Original filename,DL1_FOGALE_20220727MOM.dat
Timestamp,2022-07-27 09:13:08
Threshold,7 microns
Number of corrections,H = 6 / V = 6 / total = 12
Maximum correction amplitude,12 microns


In [5]:
# Nuestra intención es extraer la información de "Timestamp" y "Delay line number" para agregarlas como columnas nuevas

test_pd=tablas[0].T[["Timestamp","Delay line number"]] # Transponemos las filas que nos interesan de la tabla
test_pd['Timestamp'] =pd.to_datetime(test_pd['Timestamp']) # Hace que los Timestamp pasen a formato datetime64 (formato de fecha y hora estandarizado) para que se puedan buscar por tiempos en pandas
test_pd

,Timestamp,Delay line number
File info,2022-07-27 09:13:08,1


In [6]:
test_pd["Timestamp"]

File info   2022-07-27 09:13:08
Name: Timestamp, dtype: datetime64[ns]

In [7]:
tablas[1]

Rail Corrections (microns)         
                   horizontal vertical
16                         -8        0
62                        -11        0
69                          0        7
70                          0        8
71                          0        7
73                         10        0
74                          9        0
75                          8        0
78                          0        7
80                        -12        8
81                          0        8

In [8]:
n_repeat=tablas[1].shape[0] # Número de filas que tiene tablas[1]
test_pd_repeated=pd.concat([test_pd]*n_repeat).reset_index(drop=True) # Concatenamos el dataframe a que se repita la misma cantidad de veces que el n° de correcciones y limpiamos el índice
test_pd_repeated

,Timestamp,Delay line number
0,2022-07-27 09:13:08,1
1,2022-07-27 09:13:08,1
2,2022-07-27 09:13:08,1
3,2022-07-27 09:13:08,1
4,2022-07-27 09:13:08,1
5,2022-07-27 09:13:08,1
6,2022-07-27 09:13:08,1
7,2022-07-27 09:13:08,1
8,2022-07-27 09:13:08,1
9,2022-07-27 09:13:08,1


In [9]:
tablas[1].droplevel(0, axis=1) # Eliminamos fila "Rail Corrections (microns)"

,horizontal,vertical
16,-8,0
62,-11,0
69,0,7
70,0,8
71,0,7
73,10,0
74,9,0
75,8,0
78,0,7
80,-12,8


In [10]:
tablas[1].droplevel(0, axis=1).rename_axis("Rail number") # Renombramos a "Rail number"

,horizontal,vertical
Rail number,,
16,-8,0
62,-11,0
69,0,7
70,0,8
71,0,7
73,10,0
74,9,0
75,8,0
78,0,7


In [11]:
tablas[1].droplevel(0, axis=1).rename_axis("Rail number").reset_index(drop=False)
# Eliminamos el nivel 0 del índice de columnas, Cambiamos el nombre del índice a "Rail number" y lo movemos de columna

,Rail number,horizontal,vertical
0,16,-8,0
1,62,-11,0
2,69,0,7
3,70,0,8
4,71,0,7
5,73,10,0
6,74,9,0
7,75,8,0
8,78,0,7
9,80,-12,8


In [12]:
# Dado que la humedad del tunel es un elemento del archivo html y no forma parte de las tablas, leemos esta información con BeautifulSoup

from bs4 import BeautifulSoup
import re

with open(path, "r", encoding="utf-8") as f:
    html_content = f.read()

soup = BeautifulSoup(html_content, "lxml")

# "Tunnel Relative Humidity" se encuentra como la etiqueta h3
h3_tags = soup.find_all("h3")
h3_texts = [tag.text for tag in h3_tags]

# Extraemos el porcentaje de humedad del texto
for tag in h3_tags:
    texto = tag.get_text()
    match = re.search(r"(\d+(\.\d+)?%)", texto)
    porcentaje = match.group(1)

# Creamos una tabla 
porc_pd_repeated = pd.DataFrame({"Tunnel Relative Humidity": [porcentaje] * n_repeat})
porc_pd_repeated

,Tunnel Relative Humidity
0,20.1%
1,20.1%
2,20.1%
3,20.1%
4,20.1%
5,20.1%
6,20.1%
7,20.1%
8,20.1%
9,20.1%


In [13]:
pd_result=pd.concat([test_pd_repeated, porc_pd_repeated, tablas[1].droplevel(0, axis=1).rename_axis('Rail number').reset_index(drop=False)], axis=1)
# Concatenamos ambas tablas anteriormente realizadas
pd_result

,Timestamp,Delay line number,Tunnel Relative Humidity,Rail number,horizontal,vertical
0,2022-07-27 09:13:08,1,20.1%,16,-8,0
1,2022-07-27 09:13:08,1,20.1%,62,-11,0
2,2022-07-27 09:13:08,1,20.1%,69,0,7
3,2022-07-27 09:13:08,1,20.1%,70,0,8
4,2022-07-27 09:13:08,1,20.1%,71,0,7
5,2022-07-27 09:13:08,1,20.1%,73,10,0
6,2022-07-27 09:13:08,1,20.1%,74,9,0
7,2022-07-27 09:13:08,1,20.1%,75,8,0
8,2022-07-27 09:13:08,1,20.1%,78,0,7
9,2022-07-27 09:13:08,1,20.1%,80,-12,8


In [14]:
# Como Timestamp esta en formato datetime64 se puede buscar en pandas entre dos fechas en la columna
day="26"
mes="07"
año="2022"

day_2="28"
mes_2="07"
año_2="2022"


pd_result[(pd_result["Timestamp"] > año+"-"+mes+"-"+day+" 00:00:00") & (pd_result["Timestamp"] < año_2+"-"+mes_2+"-"+day_2+" 00:00:00")]
# Por ahora, solo se muestra la información de la primera tabla, de la primera fecha (único html trabajado)


,Timestamp,Delay line number,Tunnel Relative Humidity,Rail number,horizontal,vertical
0,2022-07-27 09:13:08,1,20.1%,16,-8,0
1,2022-07-27 09:13:08,1,20.1%,62,-11,0
2,2022-07-27 09:13:08,1,20.1%,69,0,7
3,2022-07-27 09:13:08,1,20.1%,70,0,8
4,2022-07-27 09:13:08,1,20.1%,71,0,7
5,2022-07-27 09:13:08,1,20.1%,73,10,0
6,2022-07-27 09:13:08,1,20.1%,74,9,0
7,2022-07-27 09:13:08,1,20.1%,75,8,0
8,2022-07-27 09:13:08,1,20.1%,78,0,7
9,2022-07-27 09:13:08,1,20.1%,80,-12,8


In [15]:
pd_result['Timestamp'] < año_2+"-"+mes_2+"-"+day_2+" 00:00:00"

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
Name: Timestamp, dtype: bool

In [16]:
pd_result['Timestamp'] > año+"-"+mes+"-"+day+" 00:00:00"

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
Name: Timestamp, dtype: bool